## Module

In [1]:
import optuna
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import log_loss
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

## Dataset

In [2]:
train = pd.read_csv('../input/titanic/train.csv')
print(train.shape)
train.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test = pd.read_csv('../input/titanic/test.csv')
print(test.shape)
test.head()

(418, 11)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
submission = pd.read_csv('../input/titanic/gender_submission.csv')
submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


## Info

In [5]:
train['Survived'].value_counts()

0    549
1    342
Name: Survived, dtype: int64

## Preprocess

In [6]:
features = [
    'Pclass',
    'Sex',
    'Age',
    'SibSp',
    'Parch',
    'Fare',
    'Embarked'
]

target = 'Survived'

In [7]:
for col in ['Sex', 'Embarked']:
    le = LabelEncoder()
    le.fit(train[col].fillna('nan'))
    
    train[col] = le.transform(train[col].fillna('nan'))
    test[col] = le.transform(test[col].fillna('nan'))

In [8]:
X_train = train[features]
y_train = train[target]
print(X_train.shape)
print(y_train.shape)

(891, 7)
(891,)


## Params

In [9]:
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.3, stratify=y_train, random_state=123)

print(X_tr.shape)
print(y_val.shape)
print(X_tr.shape)
print(y_val.shape)

(623, 7)
(268,)
(623, 7)
(268,)


In [10]:
def objective(trial):
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'silent': 1,
        'random_state': 71,
        'eta': trial.suggest_uniform('eta', 0.05, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_uniform('min_child_weight', 0.1, 10),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 0.95),
        'subsample': trial.suggest_uniform('subsample', 0.6, 0.95)
    }
    
    dtrain = xgb.DMatrix(X_tr, label=y_tr)
    dvalid = xgb.DMatrix(X_val, label=y_val)
    
    watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    num_round = 500
    
    model = xgb.train(
        params,
        dtrain,
        num_round,
        evals=watchlist,
        early_stopping_rounds=20
    )
    
    y_pred_val = model.predict(dvalid, ntree_limit=model.best_ntree_limit)
    score = log_loss(y_val, y_pred_val)
    return score

In [11]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=40)

[I 2023-10-27 03:41:51,965] A new study created in memory with name: no-name-0f7c9ff4-7352-48d5-94fa-7304b4cb663c


[03:41:51] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-logloss:0.66087	eval-logloss:0.66226
[1]	train-logloss:0.63183	eval-logloss:0.63388
[2]	train-logloss:0.60867	eval-logloss:0.61376
[3]	train-logloss:0.58614	eval-logloss:0.59287
[4]	train-logloss:0.56761	eval-logloss:0.57618
[5]	train-logloss:0.55102	eval-logloss:0.56056
[6]	train-logloss:0.53498	eval-logloss:0.54648
[7]	train-logloss:0.52247	eval-logloss:0.53673
[8]	train-logloss:0.50989	eval-logloss:0.52489
[9]	train-logloss:0.50227	eval-logloss:0.51881
[10]	train-logloss:0.49191	eval-logloss:0.51054
[11]	train-logloss:0.48686	eval-logloss:0.50505
[12]	train-logloss:0.47837	eval-logloss:0.49901
[13]	train-logloss:0.47022	eval-logloss:0.49191
[14]	train-l

[I 2023-10-27 03:41:52,238] Trial 0 finished with value: 0.4277930896757032 and parameters: {'eta': 0.07744067519636624, 'max_depth': 8, 'min_child_weight': 8.458230910952071, 'colsample_bytree': 0.9002809661679648, 'subsample': 0.8965381085744438}. Best is trial 0 with value: 0.4277930896757032.


[03:41:52] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-logloss:0.65799	eval-logloss:0.66014
[1]	train-logloss:0.62635	eval-logloss:0.62979
[2]	train-logloss:0.59955	eval-logloss:0.60722
[3]	train-logloss:0.57448	eval-logloss:0.58285
[4]	train-logloss:0.55312	eval-logloss:0.56265
[5]	train-logloss:0.53536	eval-logloss:0.54661
[6]	train-logloss:0.51837	eval-logloss:0.53097
[7]	train-logloss:0.50419	eval-logloss:0.51930
[8]	train-logloss:0.49120	eval-logloss:0.50678
[9]	train-logloss:0.48220	eval-logloss:0.50101
[10]	train-logloss:0.47107	eval-logloss:0.49055
[11]	train-logloss:0.46334	eval-logloss:0.48618
[12]	train-logloss:0.45413	eval-logloss:0.47914
[13]	train-logloss:0.44540	eval-logloss:0.47264
[14]	train-l

[I 2023-10-27 03:41:52,499] Trial 1 finished with value: 0.4172926498994009 and parameters: {'eta': 0.08117818483929862, 'max_depth': 5, 'min_child_weight': 4.432113391500656, 'colsample_bytree': 0.912120550273728, 'subsample': 0.9372819661753602}. Best is trial 1 with value: 0.4172926498994009.


...

[I 2023-10-27 03:42:05,243] Trial 37 finished with value: 0.425464682475622 and parameters: {'eta': 0.05858148386307203, 'max_depth': 5, 'min_child_weight': 4.946534475221827, 'colsample_bytree': 0.7186447908729223, 'subsample': 0.6628215913212495}. Best is trial 9 with value: 0.41296874380100573.


[03:42:05] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-logloss:0.66714	eval-logloss:0.67273
[1]	train-logloss:0.64272	eval-logloss:0.64891
[2]	train-logloss:0.62102	eval-logloss:0.63153
[3]	train-logloss:0.59994	eval-logloss:0.61208
[4]	train-logloss:0.58106	eval-logloss:0.59404
[5]	train-logloss:0.56443	eval-logloss:0.58003
[6]	train-logloss:0.54771	eval-logloss:0.56534
[7]	train-logloss:0.53414	eval-logloss:0.55422
[8]	train-logloss:0.52301	eval-logloss:0.54521
[9]	train-logloss:0.51633	eval-logloss:0.54018
[10]	train-logloss:0.50491	eval-logloss:0.53062
[11]	train-logloss:0.49833	eval-logloss:0.52402
[12]	train-logloss:0.48844	eval-logloss:0.51682
[13]	train-logloss:0.47800	eval-logloss:0.50769
[14]	train-l

[I 2023-10-27 03:42:05,570] Trial 38 finished with value: 0.41724117625671536 and parameters: {'eta': 0.058549329934156576, 'max_depth': 5, 'min_child_weight': 2.3168544117977636, 'colsample_bytree': 0.7208730882439159, 'subsample': 0.9248284527129568}. Best is trial 9 with value: 0.41296874380100573.


[03:42:05] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-logloss:0.66189	eval-logloss:0.66642
[1]	train-logloss:0.63111	eval-logloss:0.63562
[2]	train-logloss:0.60834	eval-logloss:0.61497
[3]	train-logloss:0.58514	eval-logloss:0.59424
[4]	train-logloss:0.56740	eval-logloss:0.57837
[5]	train-logloss:0.55127	eval-logloss:0.56235
[6]	train-logloss:0.53560	eval-logloss:0.54720
[7]	train-logloss:0.52207	eval-logloss:0.53599
[8]	train-logloss:0.51275	eval-logloss:0.52854
[9]	train-logloss:0.50762	eval-logloss:0.52259
[10]	train-logloss:0.49904	eval-logloss:0.51498
[11]	train-logloss:0.49481	eval-logloss:0.51033
[12]	train-logloss:0.48774	eval-logloss:0.50698
[13]	train-logloss:0.48048	eval-logloss:0.50043
[14]	train-l

[I 2023-10-27 03:42:05,848] Trial 39 finished with value: 0.439530704934984 and parameters: {'eta': 0.08522072009617665, 'max_depth': 4, 'min_child_weight': 7.858078028488696, 'colsample_bytree': 0.7750092042052554, 'subsample': 0.6176295198651403}. Best is trial 9 with value: 0.41296874380100573.


In [12]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'silent': 1,
    'random_state': 71
}

for i, j in study.best_params.items():
    params[i] = j

params

{'objective': 'binary:logistic',
 'eval_metric': 'logloss',
 'silent': 1,
 'random_state': 71,
 'eta': 0.07185159768996707,
 'max_depth': 4,
 'min_child_weight': 1.0828754685538413,
 'colsample_bytree': 0.939433173711362,
 'subsample': 0.8285990125292781}

## Evaluate

In [13]:
scores = []

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=456)

for tr_idx, val_idx in kf.split(X_train, y_train):
    X_tr, X_val = X_train.iloc[tr_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[tr_idx], y_train.iloc[val_idx]
    
    dtrain = xgb.DMatrix(X_tr, label=y_tr)
    dvalid = xgb.DMatrix(X_val, label=y_val)
    
    watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    num_round = 500
    
    model = xgb.train(
        params,
        dtrain,
        num_round,
        evals=watchlist,
        early_stopping_rounds=20
    )
    
    y_pred_val = model.predict(dvalid, ntree_limit=model.best_ntree_limit)
    y_pred_val = (y_pred_val > 0.5).astype(int)
    score = f1_score(y_val, y_pred_val, average='macro')
    scores.append(score)

[03:42:15] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-logloss:0.66160	eval-logloss:0.66126
[1]	train-logloss:0.63430	eval-logloss:0.63282
[2]	train-logloss:0.60908	eval-logloss:0.60648
[3]	train-logloss:0.58922	eval-logloss:0.58425
[4]	train-logloss:0.57054	eval-logloss:0.56410
[5]	train-logloss:0.55851	eval-logloss:0.55344
[6]	train-logloss:0.54950	eval-logloss:0.54530
[7]	train-logloss:0.53379	eval-logloss:0.52892
[8]	train-logloss:0.51995	eval-logloss:0.51501
[9]	train-logloss:0.51292	eval-logloss:0.50841
[10]	train-logloss:0.49958	eval-logloss:0.49366
[11]	train-logloss:0.48822	eval-logloss:0.48203
[12]	train-logloss:0.47850	eval-logloss:0.47327
[13]	train-logloss:0.46939	eval-logloss:0.46310
[14]	train-l

In [14]:
print('===CV scores===')
print(scores)
print(np.mean(scores))

===CV scores===
[0.8384962406015037, 0.7872485742581019, 0.8292214912280701, 0.8323039882759342, 0.7308246878846492]
0.8036189964496518


## Model

In [15]:
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.3, stratify=y_train, random_state=123)

print(X_tr.shape)
print(y_val.shape)
print(X_tr.shape)
print(y_val.shape)

(623, 7)
(268,)
(623, 7)
(268,)


In [16]:
dtrain = xgb.DMatrix(X_tr, label=y_tr)
dvalid = xgb.DMatrix(X_val, label=y_val)
    
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
num_round = 500
    
model = xgb.train(
    params,
    dtrain,
    num_round,
    evals=watchlist,
    early_stopping_rounds=20
)

[03:42:31] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-logloss:0.66190	eval-logloss:0.66431
[1]	train-logloss:0.63135	eval-logloss:0.63444
[2]	train-logloss:0.60543	eval-logloss:0.61320
[3]	train-logloss:0.58035	eval-logloss:0.59114
[4]	train-logloss:0.56012	eval-logloss:0.57291
[5]	train-logloss:0.54280	eval-logloss:0.55729
[6]	train-logloss:0.52632	eval-logloss:0.54255
[7]	train-logloss:0.51087	eval-logloss:0.53033
[8]	train-logloss:0.49679	eval-logloss:0.51648
[9]	train-logloss:0.48983	eval-logloss:0.51136
[10]	train-logloss:0.47832	eval-logloss:0.50187
[11]	train-logloss:0.47234	eval-logloss:0.49673
[12]	train-logloss:0.46210	eval-logloss:0.48898
[13]	train-logloss:0.45216	eval-logloss:0.48073
[14]	train-l

## Submit

In [17]:
X_test = test[features]

In [18]:
dtest = xgb.DMatrix(X_test)
y_pred_test = model.predict(dtest, ntree_limit=model.best_ntree_limit)
y_pred_test = (y_pred_test > 0.5).astype(int)

In [19]:
submit = pd.DataFrame(y_pred_test, index=test['PassengerId'], columns=['Survived'])
submit.to_csv('submit.csv')